![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and Google `flan-ul2` to analyze eXtensive Business Reporting Language (XBRL) tags of financial reports

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support of tag entity extraction in watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to use `flan-ul2` model to analyze XBRL tags of financial phrases.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Data loading](#data)
- [Foundation Models on watsonx](#models)
- [Model testing](#predict)
- [Score](#score)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/admin/create-services.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).


### Install and import the `datasets` and dependecies

In [ ]:
!pip install datasets | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install "ibm-watsonx-ai>=1.0.310" | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [6]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [7]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Data loading

Download the `nlpaueb/finer-139` dataset.

In [ ]:
from datasets import load_dataset

finer_train = load_dataset("nlpaueb/finer-139", split="train")

Retrieve the entity tags.

In [9]:
ner_tags = finer_train.features["ner_tags"].feature.names[5:17]
ner_tags

['B-AmortizationOfFinancingCosts',
 'B-AmortizationOfIntangibleAssets',
 'I-AmortizationOfIntangibleAssets',
 'B-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'I-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'B-AreaOfRealEstateProperty',
 'I-AreaOfRealEstateProperty',
 'B-AssetImpairmentCharges',
 'B-BusinessAcquisitionEquityInterestsIssuedOrIssuableNumberOfSharesIssued',
 'B-BusinessAcquisitionPercentageOfVotingInterestsAcquired',
 'I-BusinessAcquisitionPercentageOfVotingInterestsAcquired',
 'B-BusinessCombinationAcquisitionRelatedCosts']

Transfer the tokens into sequences for the model.

In [10]:
tokens = finer_train["tokens"][5:17]

In [11]:
sequences = [" ".join(token) for token in tokens]

Inspect exemplary sequence.

In [12]:
import json

print(json.dumps(sequences[3], indent=2))

"The Company \u2019 s reportable homebuilding segments and all other homebuilding operations not required to be reported separately have homebuilding divisions located in : East : Florida , Georgia , Maryland , New Jersey , North Carolina , South Carolina and Virginia Central : Arizona , Colorado and Texas ( 1 ) West : California and Nevada Houston : Houston , Texas Other : Illinois , Minnesota , Oregon , Tennessee and Washington ( 1 ) Texas in the Central reportable segment excludes Houston , Texas , which is its own reportable segment . Operations of the Lennar Financial Services segment include primarily mortgage financing , title insurance and closing services for both buyers of the Company \u2019 s homes and others ."


<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All available models are presented under ModelTypes class.
For more information refer to [documentation](https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#ibm_watsonx_ai.foundation_models.utils.enums.ModelTypes).


In [13]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2']


You need to specify `model_id` that will be used for inferencing:

In [14]:
model_id = ModelTypes.FLAN_UL2

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to [documentation](https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#metanames.GenTextParamsMetaNames).

In [15]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy"
}

### Initialize the model
Initialize the `Model` class with previous set params.

In [16]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

### Model's details

In [17]:
model.get_details()

{'model_id': 'google/flan-ul2',
 'label': 'flan-ul2-20b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'short_description': 'flan-ul2 is an encoder decoder model based on the T5 architecture and instruction-tuned using the Fine-tuned LAnguage Net.',
 'long_description': 'flan-ul2 (20B) is an encoder decoder model based on the T5 architecture and instruction-tuned using the Fine-tuned LAnguage Net (FLAN). Compared to the original UL2 model, flan-ul2 (20B) is more usable for few-shot in-context learning because it was trained with a three times larger receptive field. flan-ul2 (20B) outperforms flan-t5 (11B) by an overall relative improvement of +3.2%.',
 'task_ids': ['question_answering',
  'summarization',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'quality': 4}},
  {'id': 'summarization', 'ratings': {'quality': 4}},
  {'id': 'classification', 'ratings': {'quality': 4}},
  {'id': 'extraction', 'ratings': {'quality': 4}

<a id="predict"></a>
## Analyze the sentiment

Define instructions for the model. 

**HINT:** All possible tags must be attached in the instruction

In [18]:
instruction = "Determine the eXtensive Business Reporting Language tag in the financial report from following tags: B-AmortizationOfFinancingCosts, B-AmortizationOfIntangibleAssets, I-AmortizationOfIntangibleAssets, B-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount, I-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount, B-AreaOfRealEstateProperty, I-AreaOfRealEstateProperty, B-AssetImpairmentCharges, B-BusinessAcquisitionEquityInterestsIssuedOrIssuableNumberOfSharesIssued, B-BusinessAcquisitionPercentageOfVotingInterestsAcquired, B-BusinessCombinationAcquisitionRelatedCosts.\n"
print(json.dumps(instruction, indent=2))

"Determine the eXtensive Business Reporting Language tag in the financial report from following tags: B-AmortizationOfFinancingCosts, B-AmortizationOfIntangibleAssets, I-AmortizationOfIntangibleAssets, B-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount, I-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount, B-AreaOfRealEstateProperty, I-AreaOfRealEstateProperty, B-AssetImpairmentCharges, B-BusinessAcquisitionEquityInterestsIssuedOrIssuableNumberOfSharesIssued, B-BusinessAcquisitionPercentageOfVotingInterestsAcquired, B-BusinessCombinationAcquisitionRelatedCosts.\n"


Prepare few-shot examples.

In [19]:
few_shot_input = []
few_shot_target = []
singleoutput= []

for i,tl in enumerate(zip(sequences, ner_tags)):
    if (i+1)%5==0:
        singleoutput.append(f"\treport:\t{tl[0]}\n\ttag:")
        few_shot_input.append("".join(singleoutput))
        few_shot_target.append(tl[1])
        singleoutput = []
    else:
        singleoutput.append(f"\treport:\t{tl[0]}\n\ttag: {tl[1]}\n")

        

In [20]:
print(json.dumps(print(few_shot_input[0]), indent=2))

	report:	The Company ’ s reportable segments consist of : ( 1 ) Homebuilding East ( 2 ) Homebuilding Central ( 3 ) Homebuilding West ( 4 ) Homebuilding Houston ( 5 ) Lennar Financial Services ( 6 ) Rialto ( 7 ) Lennar Multifamily In the first quarter of 2016 , the Company made the decision to divide the Southeast Florida operating division into two operating segments to maximize operational efficiencies given the continued growth of the division .
	tag: B-AmortizationOfFinancingCosts
	report:	As a result of this change in management structure , the Company re - evaluated its reportable segments and determined that neither operating segment met the reportable criteria set forth in Accounting Standards Codification ( " ASC " ) 280 , Segment Reporting .
	tag: B-AmortizationOfIntangibleAssets
	report:	All prior year segment information has been restated to conform with the 2016 presentation .
	tag: I-AmortizationOfIntangibleAssets
	report:	The Company ’ s reportable homebuilding segments a

### Analyze financial phrase eXtensive Business Reporting Language using Google `flan-ul2` model.




Analyze the sentiment.

In [ ]:
results = []
for inp in few_shot_input:
    results.append(model.generate(" ".join([instruction, inp]))["results"][0])

Explore model output.

In [22]:
print(json.dumps(results, indent=2))

[
  {
    "generated_text": "B-AreaOfRealEstateProperty",
    "generated_token_count": 15,
    "input_token_count": 959,
    "stop_reason": "EOS_TOKEN"
  },
  {
    "generated_text": "B-BusinessCombinationAcquisitionRelatedCosts",
    "generated_token_count": 18,
    "input_token_count": 1126,
    "stop_reason": "EOS_TOKEN"
  }
]


<a id="score"></a>
## Score the model

**Note:** To run the Score section for model scoring please transform following `markdown` cells to `code` cells.
Have in mind that the score is calculated only on dataset sample, for relevant performance metric please score the model on the whole `nlpaueb/finer-139` dataset.

Get the true labels.

```
y_true = few_shot_target
```

Get the sentiment labels returned byt the `flan-ul2` model.

```
y_pred = [res["generated_text"] for res in results]
```

Calculate accuracy score.

```
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred, y_true))
```

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to predict the financial phrases XBRL tag with Google's `flan-ul2` on watsonx. 
 
 Check out our _[Online Documentation](https://ibm.github.io/watsonx-ai-python-sdk/samples.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.